# 08e — GPU Parallelization (Dask-CUDA & RAPIDS)

**Optional**; requires Linux + NVIDIA GPU + CUDA + RAPIDS stack. Cells will skip gracefully if imports fail.

In [ ]:
try:
    import dask_cudf, cudf, cupy
    from dask_cuda import LocalCUDACluster
    from dask.distributed import Client
    gpu_ok = True
except Exception as e:
    print('GPU libraries not available:', e)
    gpu_ok = False

In [ ]:
if gpu_ok:
    cluster = LocalCUDACluster()
    client = Client(cluster)
    client
else:
    print('Skipping cluster creation.')

In [ ]:
if gpu_ok:
    import numpy as np
    n_rows = 1_000_00  # 100k for a light demo
    data = {
        'value': np.random.random(n_rows)*200,
        'category': np.random.choice(['A','B','C','D'], size=n_rows)
    }
    gdf = cudf.DataFrame(data)
    ddf = dask_cudf.from_cudf(gdf, npartitions=4)
    res = ddf[ddf['value']>100].groupby('category').mean().compute()
    print(res)
else:
    print('cuDF demo skipped')

In [ ]:
if gpu_ok:
    import dask.array as da
    x = da.random.random((10000,10000), chunks=(10000,10000)).map_blocks(cupy.asarray)
    y = x.dot(x.T)
    out = y.sum().compute()
    float(out)
else:
    print('CuPy demo skipped')